# Collection from aisixiang.com - political/philosophical scientific articles

As of 25 Oct 2024 website has enabled some security and is blocking occassionally. Increased wait times but may want to explore other options too

This script was written for mass collection of all articles in given catagories (to train a classifier). It is this not for refined searches, but could be used as the basis for such (in time will look at doing myself).

In [1]:
import pandas as pd

import bs4
from bs4 import BeautifulSoup as soup

from urllib.request import urlopen as uReq
import urllib.request
from fake_useragent import UserAgent

import ssl
import time
import random

# Get collection URLS etc

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
ua = UserAgent() 
user_agent = ua.random
headers = {'User-Agent': user_agent}

In [4]:
base_url = 'https://www.aisixiang.com'
search_url = "/data/search?column=" #n
pagination = '&page=' #n

# Note recent structure change as of Oct 2024
# General structure (for future expansion of functionality): #https://www.aisixiang.com/data/search?searchfield=&keywords=&column=678

In [5]:
# This requires some website recon, maybe make a dict, to make more automatic for end users
lanmu_list = ['677', '678', '676', '682', '683', '680', '681', '684', '679', '685', '688', '703', '482'] # econ, socio[1], peda[1]

# Just sample, in progress
lanmu_dict = { 
    "macroeconomics": "677",
    "microeconomics": "678"
}


In [6]:
def make_request(url):
    page = urllib.request.Request(url, headers=headers) 
    uClient = uReq(page)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, 'lxml')
    return page_soup

In [7]:
def get_details(page_soup):

    results = page_soup.find('div', class_= 'search_list')
    result_list = results.find('ul').find_all('li')
    
    for result in result_list:
        url_title = result.find_all('a')
        url_title = url_title[1:]

        url = url_title[0]['href']
        url = base_url + url
        url_list.append(url)

        title = url_title[0].text
        title_list.append(title)

        date = result.find('span').text
        date_list.append(date)


In [8]:
# HTTPError: HTTP Error 429: Too Many Requests
# New issue with next cell code: they must have added some limits
# I've increased and randomize the wait, but only limited testing

In [9]:
# for testing 
lanmu_list = ['678']
lanmu_dict = { 
    "microeconomics": "678"
}

In [10]:
max_date =  '2022-05-06'
nap_range = [3,5]

In [11]:
url_list = []
title_list = []
date_list = []

counter = 1
#max_page = 2
#for lanmu in lanmu_list:
for key, value in lanmu_dict.items():
    print('Searching: ' + key)
    initial_page = base_url + search_url + value + pagination + str(counter)
    page_soup = make_request(initial_page)

    #max page
    page_list = page_soup.find('div', class_= 'list_page')
    n_pages = page_list.find_all('a') 
    max_page = int(n_pages[-2]['href'].split('=')[-1]) #probably resiliant but might be able to make more robust to changes
    print('Pages: ' + str(max_page))

    while counter <= max_page:
        search_page = base_url + search_url + value + pagination + str(counter)
        page_soup = make_request(search_page)
        print('Searching ' + search_page)
        
        get_details(page_soup)
        
        if date_list[-1] < max_date:
            print('Oldest date reached')
            break

        if counter == max_page:
            counter = 0
            print('Next catagory ... ')
            break
        
        counter += 1
        
        nap_time = random.uniform(nap_range[0], nap_range[1])
        print('Waiting: ' + str(nap_time))
        time.sleep(nap_time)

df = pd.DataFrame(list(zip(url_list, date_list, title_list)), columns =['url', 'date', 'title'])

Searching: microeconomics
Pages: 7
Searching https://www.aisixiang.com/data/search?column=678&page=1
Waiting: 3.0317451541176226
Searching https://www.aisixiang.com/data/search?column=678&page=2
Oldest date reached


In [12]:
len(title_list)

60

In [13]:
df.tail()

,url,date,title
55,https://www.aisixiang.com/data/132695.html,2022-04-12 23:47,刘平青 李楠竹：领导者的自我沉思：“现在我”、“过去我”和“
56,https://www.aisixiang.com/data/132694.html,2022-04-12 23:41,刘平青 王雪 史俊熙：城镇化背景下农民工的需求满足程度及其对
57,https://www.aisixiang.com/data/132693.html,2022-04-12 23:19,刘平青 任昆：领导者的自我沉思：“不知道”“不愿意”和“不能
58,https://www.aisixiang.com/data/132681.html,2022-04-12 23:04,王雪 刘平青：进城务工人员婚姻家庭亲密关系疏离问题研究
59,https://www.aisixiang.com/data/132641.html,2022-04-12 00:53,史俊熙 刘平青：北京创业青年：群体特征与发展状况


In [14]:
PATH = './DATA/URLS/'

df.to_csv(PATH + 'aisixiang_urls.csv', index = False)

# Load here

In [15]:
# LOAD
PATH = './DATA/URLS/'
df = pd.read_csv(PATH + 'aisixiang_urls.csv')

In [21]:
# Filtering 
# I've added in scrape to stop on page where oldest date found, but we till collect full page
df = df[df['date'] >= '2023-01-01']

In [22]:
len(df)

26

# Get Full Texts

In [ ]:
def connect(index, url):

    print('Fetching ' + str(index))
    page_soup = make_request(url)        
    
    nap_time = random.uniform(.2, 1.5)
    print('Waiting: ' + str(nap_time))
    time.sleep(nap_time)

    text = page_soup.find('div', id='content')
    text_list.append(text.text) 
    #counter +=1

In [ ]:
counter = 1
text_list = []

for index, row in df.iterrows():
    try:
        connect(index, row.url)
    except:
        print('Connection error...sleeping for 1 minute')
        time.sleep(60)
        print('Retrying...')
        connect(index, row.url)

print(len(text_list))

In [ ]:
df['text'] = text_list 

In [ ]:
df.reset_index(inplace=True)

In [ ]:
OUT_PATH = './DATA/'

df.to_csv(OUT_PATH + 'aisixiang_full.csv', index = False)